This notebook demonstrates how to use weak labeling for semi-supervised learning. We will explore different weak labeling techniques and evaluate their performance.

The weak labeling techniques we will explore are:

1. Logistic Regression
2. KNeighbors
3. Random Forest
4. Neural Network
5. Support Vector Machine

We will use the `sentence-transformers` library to generate sentence embeddings and the `scikit-learn` library to train the weak labeling models.

The weak labeling models will be trained on the labeled development set and will then be used to predict the labels of the unlabeled development set. We will then evaluate the performance of the weak labeling models on the validation set, which is also labeled.

Finally, we will save the best weak labeling models so that they can be used in the semi-supervised learning phase of the weak labeling together with the provided pipeline for inference.

## Setup

In [ ]:
import os
import pickle
import random
import sys

import numpy as np
import seaborn as sns
import torch
from dotenv import load_dotenv
from matplotlib import pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

sys.path.append(parent_dir)

load_dotenv()

DATA_DIR = os.getenv('DATA_DIR', 'data')
MODELS_DIR = os.getenv('MODELS_DIR', 'models')

DATA_DIR = os.path.abspath(os.path.join(parent_dir, DATA_DIR))
MODELS_DIR = os.path.abspath(os.path.join(parent_dir, MODELS_DIR))

assert DATA_DIR is not None
assert MODELS_DIR is not None

SEED = 1337


def set_seed():
    torch.use_deterministic_algorithms(True)

    random.seed(SEED)
    np.random.seed(SEED)

    torch.manual_seed(SEED)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


set_seed()

As the `sentence-transformers` gives us a `torch` model, we need to check the environment to see if we can use a suitable accelerator.

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
elif torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Device:', device)

## Load Datasets

We will load the labeled development set, the unlabeled development set, and the validation set.

Here a short overview of the datasets:
- Train Dataset: Labeled development set
- Validation Dataset: Validation set
- Test Dataset: Unlabeled development set

The test dataset would not be available in a real-world scenario, but we will use it to get an idea of how well the weak labeling models perform on unseen data and thus make assumptions about the performance later on the downstream tasks.

In [ ]:
from src.data_loader import load_datasets

partitions_dir = os.path.join(DATA_DIR, 'partitions')
labelled_dev, unlabelled_dev, val_set = load_datasets(partitions_dir)
train_df = labelled_dev
y_train = train_df['label']

val_df = val_set
y_val = val_df['label']

test_df = unlabelled_dev
y_test = test_df['ground_truth']

## Sentence Embeddings

We will use the `sentence-transformers` library to generate sentence embeddings for the text data. The embedding models we will discuss are:
1. `all-MiniLM-L6-v2`: Fast and efficient model
2. `all-mpnet-base-v2`: High-quality model

All the models are pre-trained on a large corpus of text data and can generate high-quality sentence embeddings. The choice of the model depends on the trade-off between quality and efficiency. From the documentation:

> The all-mpnet-base-v2 model provides the best quality, while all-MiniLM-L6-v2 is 5 times faster and still offers good quality.

In the next section, we will compare the impact on performance of the weak labeling models when using the two different embedding models and ultimately make a decision on which model to use.


In [ ]:
def get_sentence_transformer_model(model_name, device=device):
    return SentenceTransformer(model_name).to(device)


EMBEDDING_MODELS = {
    'mini_lm': get_sentence_transformer_model("all-MiniLM-L6-v2"),
    'mpnet_base': get_sentence_transformer_model("all-mpnet-base-v2")
}

The functions below will be used to generate and save the sentence embeddings for the text data using the sentence-transformers model. The embeddings will be generated for the labeled development set, the unlabeled development set, and the validation set. The embeddings will be saved to disk so that they can be loaded later without having to regenerate them. 

If the embeddings are already generated and saved to disk, the functions will load the embeddings from disk instead of regenerating them.

In [ ]:
import time

EMBEDDINGS_FOLDER = os.path.join(DATA_DIR, 'embeddings')
os.makedirs(EMBEDDINGS_FOLDER, exist_ok=True)


def load_embeddings(filename):
    """Load the embeddings from disk."""
    with open(f'{EMBEDDINGS_FOLDER}/{filename}.pkl', 'rb') as f:
        embeddings = pickle.load(f)
    return embeddings


def generate_embeddings(model, texts, verbose=True):
    """Generate sentence embeddings using the sentence-transformers model."""
    embeddings = model.encode(texts, show_progress_bar=verbose)
    return embeddings


def save_embeddings(embeddings, filename):
    """Save the embeddings to disk."""
    with open(f'{EMBEDDINGS_FOLDER}/{filename}.pkl', 'wb') as f:
        pickle.dump(embeddings, f)


def generate_and_save_embeddings(model, texts, filename):
    """Generate and save the embeddings."""
    if os.path.exists(f'{EMBEDDINGS_FOLDER}/{filename}.pkl'):
        embeddings = load_embeddings(filename)
        return embeddings

    embeddings = generate_embeddings(model, texts)
    save_embeddings(embeddings, filename)
    return embeddings


EMBEDDING_MODEL_DATA = {}
for model_name, model in EMBEDDING_MODELS.items():
    print(f'Generating embeddings for {model_name}')

    start = time.time()

    # Mappings back to partitioning keys:
    # train -> labelled development set, 
    # val -> validation set
    # test -> unlabelled development set
    X_train = generate_and_save_embeddings(model, train_df['content'].values,
                                           f'{model_name}/labelled_dev')
    X_val = generate_and_save_embeddings(model, val_df['content'].values,
                                         f'{model_name}/validation_set')
    X_test = generate_and_save_embeddings(model, test_df['content'].values,
                                          f'{model_name}/unlabelled_dev')

    end = time.time()

    print(f'Generated embeddings for {model_name} in {end - start:.2f} seconds')
    print(
        f'Train Embeddings Shape: {X_train.shape}, Validation Embeddings Shape: {X_val.shape}, Test Embeddings Shape: {X_test.shape}')

    EMBEDDING_MODEL_DATA[model_name] = {
        'X_train': X_train,
        'X_val': X_val,
        'X_test': X_test
    }

As expected the larger model `all-mpnet-base-v2` takes more time to generate the embeddings compared to the smaller model `all-MiniLM-L6-v2`. The dimensions of the embeddings also differ, with the larger model generating embeddings of size 768 and the smaller model generating embeddings of size 384.

## Evaluation Function

We will use the following function to evaluate the performance of the weak labeling models on the validation and test sets. It will print the classification report and confusion matrix for the predictions made by the weak labeling models.

The test set would not be available in a real-world scenario, but we will use it to get an idea of how well the weak labeling models perform on unseen data and thus make assumptions about the performance later on the downstream task. This obviously introduces some bias for the decision-making process, but we will use it for the sake of the demonstration.

In [ ]:
def print_evaluation(y_true, y_pred, title, plot_cm=False):
    """Prints the evaluation metrics for a classification model."""
    print(f'\nClassification Report: {title}')
    print(classification_report(y_true, y_pred))

    if plot_cm:
        cm = confusion_matrix(y_true, y_pred)
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
        plt.title(f'Confusion Matrix: {title}')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.show()

## Comparison between Embedding Models

We discovered in the process of working on the comparison of weak labelling approaches, that the difference in the embeddings generated by the two models significantly affects the performance of the weak labeling models. For this reason, we will compare the performance of the weak labeling models trained on the embeddings generated by the two models to get an idea of which model performs better. We will make a decision afterwards on which model to use for all the subsequent approaches discussed. This as the computational cost would otherwise be excessive also considering the downstream tasks of fine-tuning and transfer learning for both embedding models used to generate weak labels. 

We will use a simple Logistic Regression model for this comparison. As this is a baseline comparison, we will not perform hyperparameter tuning for the Logistic Regression model.

In [ ]:
from sklearn.linear_model import LogisticRegression

for model_name, model_data in EMBEDDING_MODEL_DATA.items():
    X_train = model_data['X_train']
    X_val = model_data['X_val']
    X_test = model_data['X_test']

    log_reg = LogisticRegression(max_iter=100_000)
    log_reg.fit(X_train, y_train)

    print_evaluation(y_val, log_reg.predict(X_val),
                     f'Logistic Regression (model={model_name}, set=Validation Set)')

    print_evaluation(y_test, log_reg.predict(X_test),
                     f'Logistic Regression (model={model_name}, set=Test Set)')

We can see that for this baseline comparison the `all-mpnet-base-v2` model performs better than the `all-MiniLM-L6-v2` model with a considerable margin of roughly 0.06% in F1-score on both the validation and test sets.

As the `all-mpnet-base-v2` model performs better than the `all-MiniLM-L6-v2` model, we will use the embeddings generated by the `all-mpnet-base-v2` model for the subsequent steps although the dimensionality of the embeddings is higher. This could result in higher computational costs but give the observed margin in performance this seems justified.

In [ ]:
# set X_train, X_val, X_test to the embeddings of the best model for subsequent steps
X_train = EMBEDDING_MODEL_DATA['mpnet_base']['X_train']
y_train = train_df['label']

X_val = EMBEDDING_MODEL_DATA['mpnet_base']['X_val']
y_val = val_df['label']

X_test = EMBEDDING_MODEL_DATA['mpnet_base']['X_test']
y_test = test_df['ground_truth']

print(f'X_train Shape: {X_train.shape}, y_train Shape: {y_train.shape}')
print(f'X_val Shape: {X_val.shape}, y_val Shape: {y_val.shape}')
print(f'X_test Shape: {X_test.shape}, y_test Shape: {y_test.shape}')

## Approach 1: Logistic Regression

Logistic Regression is selected as the first approach due to its simplicity and interpretability, making it an ideal baseline for weak labeling tasks. It is a well-understood algorithm that provides clear insights into the relationship between features and the target variable. Despite its simplicity, Logistic Regression is effective for binary classification tasks, such as the one in this challenge. It serves as a solid starting point to evaluate the performance of weak labeling before exploring more complex models.

We will use grid search to find the best hyperparameters for the logistic regression model as we will do for all the other subsequent approaches discussed in this notebook.

In [ ]:
from sklearn.model_selection import GridSearchCV

log_reg_param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 3, 5, 10, 100],
    'max_iter': [100_000]
}

log_reg_grid_search = GridSearchCV(LogisticRegression(random_state=SEED),
                                   log_reg_param_grid,
                                   n_jobs=-1,
                                   cv=5,
                                   verbose=3)
log_reg_grid_search.fit(X_train, y_train)

log_reg_model = log_reg_grid_search.best_estimator_
print(f'Best Parameters: {log_reg_grid_search.best_params_}')

print_evaluation(y_val, log_reg_model.predict(X_val),
                 'Logistic Regression (Validation Set)')

print_evaluation(y_test, log_reg_model.predict(X_test),
                 'Logistic Regression (Test Set)')

- **Balanced Performance**: The model shows consistent precision, recall, and F1-score of 0.86 across both classes, indicating it can accurately classify both positive and negative instances without bias.
- **High Reliability**: An accuracy of 0.86 reflects the model's strong ability to make correct predictions, making it a dependable choice for weak labeling tasks.
- **Good Generalization**: The similar performance on both validation and test sets suggests the model generalizes well to new, unseen data.

## Approach 2: KNeighbors Weak Labeling

The KNeighbors classifier is chosen for its intuitive approach to classification by considering the proximity of data points in the feature space. This method is particularly suitable for weak labeling as it leverages the notion of clustering, which is analogous to finding the nearest neighbors for a given data point. By using Grid Search to tune hyperparameters, we aim to identify the optimal configuration that can enhance the performance of the weak labeling model.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

param_grid = {
    'n_neighbors': [3, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
}

grid_search = GridSearchCV(KNeighborsClassifier(),
                           param_grid,
                           n_jobs=-1,
                           cv=5,
                           verbose=3)

grid_search.fit(X_train, y_train)
knn_model = grid_search.best_estimator_

print_evaluation(y_val, knn_model.predict(X_val),
                 'KNeighbors Weak Labeling (Validation Set)')

print_evaluation(y_test, knn_model.predict(X_test),
                 'KNeighbors Weak Labeling (Test Set)')

- **Class Detection**: Higher recall for class 1 (0.85) compared to class 0 (0.78-0.74) indicates the model is more effective at identifying positive instances, beneficial for tasks like fraud detection where identifying positives is crucial. As this is a weak labeling task, the model is expected to be better at detecting positive instances.
- **Precision-Recall Trade-off**: Slightly higher precision for class 0 suggests the model may misclassify some positive instances as negative, a trade-off that needs careful management based on task requirements.
- **Moderate Generalization**: Accuracy ranging from 0.82 to 0.79 shows reasonable performance but indicates a slight decline on the test set, suggesting the model could improve in handling unseen data.
- **Dimensionality Challenges**: The variability in performance metrics could highlight potential issues with high-dimensional data, suggesting techniques like PCA could enhance performance by focusing on key features in a lower-dimensional space.

## Approach 2.1 KNeighbors Weak Labeling on PCA reduced embeddings

To address the limitations of the KNeighbors classifier in high-dimensional spaces, we incorporate Principal Component Analysis (PCA) to reduce the dimensionality of the sentence embeddings. This step is justified by the fact that KNeighbors can struggle with high-dimensional data, leading to the curse of dimensionality. PCA helps in capturing the most significant features, thereby improving the efficiency and performance of the KNeighbors classifier in the weak labeling task.


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=8)
X_train_pca = pca.fit_transform(X_train)

grid_search.fit(X_train_pca, y_train)
knn_pca_model = grid_search.best_estimator_
print(f'Best Parameters: {grid_search.best_params_}')

print_evaluation(y_val, knn_pca_model.predict(pca.transform(X_val)),
                 'KNeighbors Weak Labeling (PCA Reduced) (Validation Set)')

print_evaluation(y_test, knn_pca_model.predict(pca.transform(X_test)),
                 'KNeighbors Weak Labeling (PCA Reduced) (Test Set)')

- **Dimensionality Reduction Impact**: Applying PCA to reduce dimensionality resulted in lower performance metrics compared to the original KNeighbors approach.
- **Class Detection**: Recall for class 1 is 0.79, while for class 0 it is 0.70, indicating better detection of positive instances but weaker performance for negative instances.
- **Overall Accuracy**: The accuracy of 0.74 suggests that while PCA helps in reducing the feature space, it might also lead to loss of important information, affecting the model's performance.
- **Balanced Metrics**: Precision, recall, and F1-score are all 0.74, indicating that the model performs equally well across different metrics but at a lower level compared to the non-PCA approach.

## Approach 3: Random Forest

Random Forest is selected for its robustness and ability to handle complex data through ensemble learning. This approach justifies its use by combining multiple decision trees to improve classification accuracy and prevent overfitting. Random Forests are particularly effective in capturing nonlinear relationships and interactions between features, making them a powerful choice.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

gradient_boosting_param_grid = {
    'n_estimators': [100, 500, 1000],
    'max_depth': [3, 5, 7, 9, 11, 13]
}

rf_grid_search = GridSearchCV(RandomForestClassifier(),
                              gradient_boosting_param_grid,
                              n_jobs=-1,
                              cv=5,
                              verbose=3)

rf_grid_search.fit(X_train, y_train)
rf_model = rf_grid_search.best_estimator_
print(f'Best Parameters: {rf_grid_search.best_params_}')

print_evaluation(y_val, rf_model.predict(X_val),
                 'Random Forest (Validation Set)')

print_evaluation(y_test, rf_model.predict(X_test),
                 'Random Forest (Test Set)')

- **Balanced Performance**: Precision, recall, and F1-score are consistently high at 0.83-0.84 for both classes, indicating balanced and accurate classification without favoring either class.
- **High Reliability**: An accuracy of 0.83 across both validation and test sets suggests that the Random Forest model is reliable and makes correct predictions in a significant majority of cases.
- **Good Generalization**: The similar performance metrics on both the validation and test sets indicate that the model generalizes well to new, unseen data, which is essential for real-world applications.
- **Effective Ensemble Learning**: The consistent performance metrics reflect the robustness of Random Forest's ensemble learning approach, which combines multiple decision trees to improve classification accuracy and prevent overfitting.

## Approach 4: Neural Network

Neural Networks are chosen for their capability to model complex patterns and relationships in the data. This approach is justified by the neural network's ability to learn from high-dimensional sentence embeddings and possibly capture intricate feature representations.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_param_grid = {
    'hidden_layer_sizes': [(1024, 512), (512, 128), (256, 64), (128, 32)],
    'max_iter': [10000],
}

mlp_grid_search = GridSearchCV(MLPClassifier(),
                               mlp_param_grid,
                               n_jobs=-1,
                               cv=5,
                               verbose=3)
mlp_grid_search.fit(X_train, y_train)
mlp_model = mlp_grid_search.best_estimator_

plt.plot(mlp_model.loss_curve_)
plt.title('Neural Network Loss Curve')
plt.xlabel('Number of Iterations')
plt.ylabel('Loss')
plt.show()

print(f'Best Parameters: {mlp_grid_search.best_params_}')

print_evaluation(y_val, mlp_model.predict(X_val),
                 'Neural Network (Validation Set)')

print_evaluation(y_test, mlp_model.predict(X_test),
                 'Neural Network (Test Set)')

- **Convergence**: The loss curve shows that the neural network converges to a stable loss value, indicating that the model has effectively learned the underlying patterns in the data but may suffer from overfitting looking solely at the loss curve.
- **Balanced Performance**: Precision, recall, and F1-score are all consistently at 0.84-0.85 for both classes, indicating the neural network's ability to accurately classify both positive and negative instances without bias.
- **High Accuracy**: An accuracy of 0.84 across both validation and test sets reflects the model's strong ability to make correct predictions, showcasing its reliability for weak labeling tasks.
- **Good Generalization**: The consistent performance on both the validation and test sets suggests that the neural network generalizes well to unseen data, an essential trait for real-world applications.
- **Effective Feature Representation**: The balanced and high performance metrics indicate that the neural network is effectively capturing complex patterns and relationships in the data, leveraging its architecture to provide robust predictions.

## Approach 5: Support Vector Machine

Support Vector Machines (SVM) are included for their effectiveness in high-dimensional spaces and robustness against overfitting. SVMs are particularly suitable for weak labeling as they can find the optimal hyperplane that maximizes the margin between different classes. This approach is justified by the SVM's strong theoretical foundation and its ability to handle both linear and nonlinear classification tasks through different kernel functions, ensuring versatility.

In [ ]:
from sklearn.svm import SVC

svm_param_grid = {
    'C': [0.0001, 0.001, 0.01, 0.1, 1, 3, 5, 10],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear', 'poly', 'sigmoid']
}

svm_grid_search = GridSearchCV(SVC(),
                               svm_param_grid,
                               n_jobs=-1,
                               cv=5,
                               verbose=3)

svm_grid_search.fit(X_train, y_train)
svm_model = svm_grid_search.best_estimator_
print(f'Best Parameters: {svm_grid_search.best_params_}')

print_evaluation(y_val, svm_model.predict(X_val),
                 'Support Vector Machine (Validation Set)')

print_evaluation(y_test, svm_model.predict(X_test),
                 'Support Vector Machine (Test Set)')

- **High Precision and Recall**: Precision and recall are both consistently at 0.86 across both classes, indicating the SVM's effectiveness in correctly identifying both positive and negative instances without bias.
- **Consistent F1-Score**: An F1-score of 0.86 for both classes shows that the model maintains a balance between precision and recall, ensuring reliable performance.
- **High Accuracy**: An accuracy of 0.86 on both validation and test sets suggests that the SVM is highly reliable and capable of making correct predictions consistently.
- **Good Generalization**: The similar performance metrics on both the validation and test sets imply that the SVM generalizes well to unseen data, making it a robust choice for real-world applications.
- **Effective Margin Maximization**: The strong and balanced performance of the SVM reflects its ability to effectively find the optimal hyperplane that maximizes the margin between classes, ensuring robust classification.

### Comparison of All Approaches

#### Logistic Regression
- **Performance**: Consistently high precision, recall, and F1-score of roughly 0.86 across both classes on validation and test sets.
- **Accuracy**: Validation set: 0.86, Test set: 0.86
- **Generalization**: Excellent, with similar metrics on validation and test sets.
- **Key Strength**: Simplicity and computational efficiency with robust generalization.

#### KNeighbors
- **Performance**: Lower recall for class 0 (0.78/0.74) but higher for class 1 (0.85 on both sets). Precision varies between classes.
- **Accuracy**: Validation set: 0.82, Test set: 0.79
- **Generalization**: Moderate, with a slight decline on the test set.
- **Key Strength**: Effective for detecting class 1 instances; most likely high-dimensional data is the limiting factor.

##### KNeighbors with PCA
- **Performance**: Reduced precision and recall compared to standard KNeighbors, with balanced but lower metrics.
- **Accuracy**: Validation set: 0.77, Test set: 0.74
- **Generalization**: Weaker compared to other models and especially standard KNeighbors.
- **Key Strength**: Dimensionality reduction; however, potential loss of important information.

#### Random Forest
- **Performance**: Balanced precision, recall, and F1-score of 0.83-0.84 across both classes.
- **Accuracy**: Validation set: 0.83, Test set: 0.83
- **Generalization**: Good, with consistent metrics on validation and test sets.
- **Key Strength**: Robust ensemble learning, capturing complex relationships in the data.

#### Neural Network
- **Performance**: Balanced precision, recall, and F1-score of 0.84-0.85 across both classes on both sets.
- **Accuracy**: Validation set: 0.84, Test set: 0.84
- **Generalization**: Good, with consistent metrics on validation and test sets.
- **Key Strength**: Effective feature representation and capturing complex patterns.

#### Support Vector Machine (SVM)
- **Performance**: Consistently high precision, recall, and F1-score of 0.86 across both classes.
- **Accuracy**: Validation set: 0.86, Test set: 0.86
- **Generalization**: Excellent, with similar metrics on validation and test sets.
- **Key Strength**: Effective margin maximization, providing robust classification.

### Summary
- **Best Performers**: Logistic Regression and SVM show the highest and most consistent performance metrics, with an accuracy of 0.86 on both the validation and test set, indicating strong generalization and reliability.
- **Good Performers**: Neural Network and Random Forest also demonstrate high performance and good generalization with slightly lower accuracy in the area around 0.83-0.84.
- **Moderate Performer**: KNeighbors, both standard and with PCA, show reasonable performance but with lower accuracy and generalization compared to other models. The PCA approach particularly suffers from potential information loss, leading to weaker performance.

Overall all weak labeling models show good performance with high accuracy and balanced metrics, indicating their reliability and effectiveness in classifying the data. To consider are the computational efficiency and generalization capabilities, with Logistic Regression and SVM standing out as the top performers in this regard.

## Save Best Models

For the semi-supervised learning phase of the weak labeling, we will save the best weak labeling models. The saved models can be used for inference in the `weak_labelling.py` script. For an example usage of the weak labeling pipeline, refer to the `README.md` file.

Although we will not all the models in the final pipeline, we will save them for reference and potential future use and completeness.

In [ ]:
MODELS_FOLDER = os.getenv('MODELS_DIR', 'models')
os.makedirs(MODELS_FOLDER, exist_ok=True)


def save_model(model, filename, overwrite=False):
    if os.path.exists(f'{MODELS_FOLDER}/weak_labelling/{filename}.pkl') and not overwrite:
        print(f'{filename} already exists. Skipping...')
        return

    with open(f'{MODELS_FOLDER}/weak_labelling/{filename}.pkl', 'wb') as f:
        pickle.dump(model, f)


save_model(log_reg_model, 'log_reg')
save_model(knn_model, 'knn')
save_model(knn_pca_model, 'knn_pca')
save_model(rf_model, 'rf')
save_model(mlp_model, 'mlp')
save_model(svm_model, 'svm')